In [278]:
import os
import math
import random
import subprocess
import numpy as np
import pandas as pd
import pandas_plink as ppl
import statsmodels.api as sm
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split

### scaled-up cis-eQTL for chromosome 21

In [ ]:
os.makedirs('ciseqtl_chr21', exist_ok=True)
os.makedirs('APP_PRS', exist_ok=True)

In [ ]:
# generate all coord txt
gene_annot = pd.read_csv('data/gene_annot.txt', sep='\t')
gene_annot_21 = gene_annot[gene_annot['CHR'] == 21]

for i, row in gene_annot_21.iterrows():
    with open(f"ciseqtl_chr21/coords/{row['ID']}_coords.txt", 'w') as f:
        f.write(f"chr{row['CHR']}\t{row['START']-500000}\t{row['STOP']+500000}\t{row['ID']}\t0\t+\n") # +- 500 kb

In [127]:
# run plink2 for all genes
for f in os.listdir('ciseqtl_chr21/coords'):
    if f.endswith('_coords.txt'):
        gene = f.split('_coords.txt')[0]
        fp = os.path.join('ciseqtl_chr21/coords', f)
        out_prefix = f'ciseqtl_chr21/snps/1000G.EUR.21.{gene}'
        command = [
            "./plink2", 
            "--bfile", "LDREF/1000G.EUR.21",
            "--extract", "bed1", fp,
            "--out", out_prefix,
            "--make-bed"
        ]
        try:
            result = subprocess.run(command, capture_output=True, text=True, check=True)
            os.remove(f'ciseqtl_chr21/snps/1000G.EUR.21.{gene}.log')
        except subprocess.CalledProcessError as e:
            print(e.stderr)

Error: No variants remaining after main filters.

Error: No variants remaining after main filters.

Error: No variants remaining after main filters.

Error: No variants remaining after main filters.

Error: No variants remaining after main filters.



In [ ]:
chr21_eqtl = []
gene_exp = pd.read_csv('data/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt', sep='\t')

In [ ]:
for f in os.listdir('ciseqtl_chr21/snps'):
    if f.endswith('.bed'):
        prefix = f.split('.bed')[0]
        gene_ID = prefix.split('.')[-1]
        print(gene_ID)
        gene_SYM = gene_annot_21[gene_annot_21['ID']==gene_ID]['SYM'].iloc[0]
        if gene_exp['TargetID'].str.startswith(gene_SYM).sum() > 0:
            (bim, fam, bed) = ppl.read_plink(f'ciseqtl_chr21/snps/{prefix}')
            fam_ids = fam['iid']
            snps = []
            y = []
            for i, iid in enumerate(fam_ids):
                if iid in gene_exp.columns:
                    snps.append(bed.compute()[:, i])
                    y.append(gene_exp[gene_exp['TargetID'].str.startswith(gene_SYM)][iid].iloc[0])
            snps = np.array(snps)

            for i in range(snps.shape[1]):
                if np.all(snps[:, i] == snps[:, i][0]):
                    print(f"SNP {bim['snp'].iloc[i]} has no variation, skipping.")
                    continue
                X = sm.add_constant(snps[:, i])
                model = sm.OLS(y, X)
                result = model.fit()
                chr21_eqtl.append({'CHR': 21, 'BP': bim['pos'].iloc[i], 'REF': bim['a0'].iloc[i], 
                                 'ALT': bim['a1'].iloc[i], 'P': result.pvalues[1], 'BETA': result.params[1],
                                'SE': result.bse[1], 'SNP': bim['snp'].iloc[i]}) 

SH3BGR


Mapping files: 100%|█████████████████████████████| 3/3 [00:00<00:00, 214.42it/s]


DYRK1A


Mapping files: 100%|█████████████████████████████| 3/3 [00:00<00:00, 440.10it/s]


MCM3AP-AS1


...

Mapping files: 100%|█████████████████████████████| 3/3 [00:00<00:00, 374.07it/s]


KCNE1
SYNJ1


Mapping files: 100%|█████████████████████████████| 3/3 [00:00<00:00, 564.86it/s]


LINC00160
MCM3AP


Mapping files: 100%|█████████████████████████████| 3/3 [00:00<00:00, 428.92it/s]


In [170]:
chr21_eqtl_df = pd.DataFrame(chr21_eqtl).sort_values(by=['CHR', 'BP'])
chr21_eqtl_df.to_csv('ciseqtl_chr21/locuszoom.txt', sep='\t', index=False)

In [184]:
chr21_eqtl_df['SNP'].nunique()

12579

In [280]:
plt.scatter(chr21_eqtl_df['BP'], -math.log10(chr21_eqtl_df['P']));

TypeError: cannot convert the series to <class 'float'>

### PRS model
gene of interest: APP

In [262]:
sample_ids = []
for f in os.listdir('ciseqtl_chr21/snps'):
    if f.endswith('APP.bed'):
        prefix = f.split('.bed')[0]
        gene_ID = prefix.split('.')[-1]
        print(gene_ID)
        gene_SYM = gene_annot_21[gene_annot_21['ID']==gene_ID]['SYM'].iloc[0]
        if gene_exp['TargetID'].str.startswith(gene_SYM).sum() > 0:
            (bim, fam, bed) = ppl.read_plink(f'ciseqtl_chr21/snps/{prefix}')
            fam_ids = fam['iid']
            snps = []
            y = []
            for i, iid in enumerate(fam_ids):
                if iid in gene_exp.columns:
                    snps.append(bed.compute()[:, i])
                    y.append(gene_exp[gene_exp['TargetID'].str.startswith(gene_SYM)][iid].iloc[0])
                    sample_ids.append(iid)
            snps = np.array(snps)

APP


Mapping files: 100%|█████████████████████████████| 3/3 [00:00<00:00, 291.15it/s]


In [252]:
# random sample
train_idx = random.sample(list(np.arange(len(y))), int(len(y) * 0.8))
remaining_idxs = [idx for idx in list(np.arange(len(y))) if idx not in train_idx]
val_idx = random.sample(remaining_idxs, len(remaining_idxs)//2)
test_idx = [idx for idx in remaining_idxs if idx not in val_idx]

In [253]:
app_eqtl_train = []
snps_train = snps[train_idx, :]
y_train = np.array(y)[train_idx]
mdl = LinearRegression()
for i in range(snps_train.shape[1]):
    if np.all(snps_train[:, i] == snps_train[:, i][0]):
        print(f"SNP {bim['snp'].iloc[i]} has no variation, skipping.")
        continue
    X = sm.add_constant(snps_train[:, i])
    model = sm.OLS(y_train, X)
    result = model.fit()
    app_eqtl_train.append({'CHR': 21, 'BP': bim['pos'].iloc[i], 'REF': bim['a0'].iloc[i], 
                     'ALT': bim['a1'].iloc[i], 'P': result.pvalues[1], 'BETA': result.params[1],
                    'SE': result.bse[1], 'SNP': bim['snp'].iloc[i]}) 
app_eqtl_df_train = pd.DataFrame(app_eqtl_train)
app_eqtl_df_train.to_csv('APP_PRS/eqtl_train.txt', sep='\t', index=False)

In [281]:
app_eqtl_df_train

,CHR,BP,REF,ALT,P,BETA,SE,SNP
0,21,25382128,T,C,0.220652,1.972576,1.606845,rs2828696
1,21,25382268,T,C,0.302136,1.663308,1.608899,rs2828697
2,21,25382938,C,T,0.634707,0.785753,1.651968,rs7275309
3,21,25382976,G,A,0.746291,-0.877380,2.709123,rs928794
4,21,25383259,T,C,0.485747,1.152201,1.650625,rs9979626
...,...,...,...,...,...,...,...,...
623,21,26662736,A,G,0.662604,-0.648922,1.485640,rs2829680
624,21,26665558,A,G,0.687600,0.608014,1.510434,rs6516661
625,21,26666834,T,C,0.454197,-1.449973,1.934569,rs2829684
626,21,26669324,A,G,0.687600,0.608014,1.510434,rs2829686


In [266]:
train_samples = [sample_ids[idx] for idx in train_idx]
with open("APP_PRS/train_samples.txt", "w") as f:
    for sample in train_samples:
        f.write(f"{sample} {sample}\n")

In [268]:
val_samples = [sample_ids[idx] for idx in val_idx]
with open("APP_PRS/val_samples.txt", "w") as f:
    for sample in val_samples:
        f.write(f"{sample} {sample}\n")

In [269]:
test_samples = [sample_ids[idx] for idx in test_idx]
with open("APP_PRS/test_samples.txt", "w") as f:
    for sample in test_samples:
        f.write(f"{sample} {sample}\n")